In [1]:
import time
import folium
import geocoder

import pandas as pd

startClk = time.time()

def calc_distance(data): # 거리 계산
    distance = []
    
    n = len(data)
    
    error = 0
    
    add = 0
    
    for i in data.index:
        
        dis_lat =  data.loc[i,'위도']
        dis_long = data.loc[i,'경도']
        
        if dis_lat == "ERROR" or dis_long == "ERROR":
            data.loc[i,'거리'] = error
        
        else :
            distance.insert(i,(((anu_lat - dis_lat) ** 2 + (anu_long - dis_long) ** 2) ** 0.5) * 100000)
            data.loc[i,'거리'] = distance[add]
            add = add + 1
    
    
    return data

def heap_sort(data):
    for i in range(1,len(data)): # 최대 힙 구하기
        left_node = i
        while left_node != 0:
            right_node = (left_node - 1) // 2
            if data.loc[right_node,'거리'] < data.loc[left_node,'거리']:
                data.loc[right_node,'거리'], data.loc[left_node,'거리'] = data.loc[left_node,'거리'], data.loc[right_node,'거리']
                data.loc[right_node,'이름'], data.loc[left_node,'이름'] = data.loc[left_node,'이름'], data.loc[right_node,'이름']
                data.loc[right_node,'위도'], data.loc[left_node,'위도'] = data.loc[left_node,'위도'], data.loc[right_node,'위도']
                data.loc[right_node,'경도'], data.loc[left_node,'경도'] = data.loc[left_node,'경도'], data.loc[right_node,'경도']
            left_node = right_node
            
    for j in reversed(data.index): # 힙 정렬  - for문 거꾸로 돌리기
        data.loc[0,'거리'], data.loc[j,'거리'] = data.loc[j,'거리'], data.loc[0,'거리']
        data.loc[0,'이름'], data.loc[j,'이름'] = data.loc[j,'이름'], data.loc[0,'이름']
        data.loc[0,'위도'], data.loc[j,'위도'] = data.loc[j,'위도'], data.loc[0,'위도']
        data.loc[0,'경도'], data.loc[j,'경도'] = data.loc[j,'경도'], data.loc[0,'경도']
    
        right_node = 0
        left_node = 1
        while left_node < j:
            left_node = 2 * right_node + 1
            if left_node < j - 1 and data.loc[left_node,'거리'] < data.loc[left_node+1,'거리']:
                left_node += 1
            if left_node < j and data.loc[right_node,'거리'] < data.loc[left_node,'거리']:
                data.loc[right_node,'거리'], data.loc[left_node,'거리'] = data.loc[left_node,'거리'], data.loc[right_node,'거리']
                data.loc[right_node,'이름'], data.loc[left_node,'이름'] = data.loc[left_node,'이름'], data.loc[right_node,'이름']
                data.loc[right_node,'위도'], data.loc[left_node,'위도'] = data.loc[left_node,'위도'], data.loc[right_node,'위도']
                data.loc[right_node,'경도'], data.loc[left_node,'경도'] = data.loc[left_node,'경도'], data.loc[right_node,'경도']
            right_node = left_node
            
    return data

def view_marker(data,num):
    error = 0
    
    n = len(data)
    
    for i in range(n): # 에러 예외 처리
        if data.loc[i,'위도'] == "ERROR" or data.loc[i,'경도'] == "ERROR":
            error = error + 1
        else:
            break
            
    error_end = 0
    error_end = error + num
    
    for i in range(error,error_end): # 정렬된 가까운 식당 30개 마커 찍기
        if data.loc[i,'거리'] == 0:
            continue
        
        sub_title = data.loc[i,'이름']
        sub_lat = data.loc[i,'위도']
        sub_long = data.loc[i,'경도']
        
        folium.Marker([sub_lat,sub_long],tooltip = sub_title).add_to(m)

excel_data = pd.read_excel('restaurant_little.xlsx', header = 0) # 액셀 데이터 프레임 형태로 불러오기

anu = geocoder.ip('me') # 현재 위도를 구하기 위한 내장 함수 
    
anu_lat = anu.lat # 현재 위도
anu_long = anu.lng # 현재 경도

calc_distance(excel_data) # 거리 계산 

m = folium.Map(location=[anu_lat, anu_long],zoom_start=16) # 현재 위치 기준으로 줌

heap_sort(excel_data) # 데이터 프레임 정렬

folium.Marker([anu_lat,anu_long],tooltip = "현재 위치").add_to(m) # 현재 위치 마커 찍기

view_marker(excel_data,30) # 정렬된 데이터 프레임을 30개까지 지도 마커로 출력
    
print("\n코드 실행 시간 : ", time.time() - startClk, "초")

m


코드 실행 시간 :  3928.5224192142487 초
